<a href="https://colab.research.google.com/github/angelaguzmanar/Marketing/blob/main/Despliegue_M.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
import sqlite3 as sql
import openpyxl


####Paquete para sistema basado en contenido ####
from sklearn.preprocessing import MinMaxScaler
from sklearn import neighbors

In [3]:
#conectamos con Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
  #### conectar_base_de_Datos#################
conn=sql.connect('/content/drive/MyDrive/Analitica3/Entrega_Marketing/db_movies')
cur=conn.cursor()

#Convertir en df
movies=pd.read_sql('select * from movies_final5', conn )
ratings=pd.read_sql('select * from ratings_final5', conn)
usuarios=pd.read_sql('select distinct (userId) as userId from ratings_final5',conn)
df_movies=pd.read_sql('select * from df_movies', conn )
df_moviesdum2=pd.read_sql('select * from df_moviesdum2', conn)


In [5]:
###############Basado en contenido todo lo visto por el usuario Knn#############################
def recomendar(userId):

    rating =pd.read_sql('select * from ratings_final5 where userId=:user',conn, params={'user':userId})
    l_movies_r=rating['movieId'].to_numpy()
    movies_r=df_moviesdum2[df_moviesdum2['movieId'].isin(l_movies_r)]
    movies_r=movies_r.drop(columns=['movieId','title'])
    movies_r["indice"]=1
    centroide=movies_r.groupby("indice").mean()

    movies_nr=df_moviesdum2[~df_moviesdum2['movieId'].isin(l_movies_r)]
    movies_nr=movies_nr.drop(columns=['movieId','title'])
    model=neighbors.NearestNeighbors(n_neighbors=11, metric='cosine')
    model.fit(movies_nr)
    dist, idlist = model.kneighbors(centroide)
    ids=idlist[0] ### queda en un array anidado, para sacarlo
    recomend_b=df_movies.loc[ids][['title','movieId']]
    leidos=df_movies[df_movies['movieId'].isin(l_movies_r)][['title','movieId']]
    return recomend_b


In [7]:
##### Generar recomendaciones para usuario lista de usuarios ####
def main(list_user):

    recomendaciones_todos=pd.DataFrame()
    for userId in list_user:

        recomendaciones=recomendar(userId)
        recomendaciones["userId"]=userId
        recomendaciones.reset_index(inplace=True,drop=True)

        recomendaciones_todos=pd.concat([recomendaciones_todos, recomendaciones])

    recomendaciones_todos.to_excel('/content/drive/MyDrive/Analitica3/Entrega_Marketing/recomendaciones.xlsx')
    recomendaciones_todos.to_csv('/content/drive/MyDrive/Analitica3/Entrega_Marketing/recomendaciones.xlsx')


if __name__=="__main__":
    list_user=[590,600,483]
    main(list_user)

In [6]:
import sys
sys.executable

'/usr/bin/python3'